# Práctica SQL - Parte 1: Base de Datos Northwind

En esta práctica vamos a trabajar con la base de datos Northwind, un ejemplo clásico de Microsoft que modela una empresa de comercio internacional.

## Esquema de la Base de Datos

![Esquema Northwind](https://miro.medium.com/v2/resize:fit:944/1*Qn-ac6Va4Oa0vLsH1suSaw.png)


Primero, instalemos las librerías necesarias:

In [1]:
!pip install ipython-sql==0.3.9 prettytable==3.5.0 sqlalchemy==1.4.46 --quiet


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
%load_ext sql
%sql sqlite:///northwind.db

'Connected: @northwind.db'

# Ejemplos de Consultas Básicas

## 1. SELECT y WHERE
Veamos los productos más caros (precio unitario mayor a $50):

In [4]:
%%sql
SELECT ProductName, UnitPrice
FROM Products
WHERE UnitPrice > 50
ORDER BY UnitPrice DESC;

 * sqlite:///northwind.db
Done.


ProductName,UnitPrice
Côte de Blaye,263.5
Thüringer Rostbratwurst,123.79
Mishi Kobe Niku,97
Sir Rodney's Marmalade,81
Carnarvon Tigers,62.5
Raclette Courdavault,55
Manjimup Dried Apples,53


## 2. LIKE
Busquemos productos que contengan la palabra "Queso" en su nombre:

In [5]:
%%sql

SELECT ProductName, UnitPrice, UnitsInStock
FROM Products
WHERE ProductName LIKE '%Queso%';

 * sqlite:///northwind.db
Done.


ProductName,UnitPrice,UnitsInStock
Queso Cabrales,21,22
Queso Manchego La Pastora,38,86


## 3. JOIN
Veamos los pedidos junto con el nombre del cliente y el empleado que lo procesó:

In [6]:
%%sql

SELECT o.OrderID, 
       c.CompanyName as Cliente,
       e.FirstName || ' ' || e.LastName as Empleado 
FROM Orders o
JOIN Customers c ON o.CustomerID = c.CustomerID
JOIN Employees e ON o.EmployeeID = e.EmployeeID
LIMIT 5;

 * sqlite:///northwind.db
Done.


OrderID,Cliente,Empleado
10248,Vins et alcools Chevalier,Steven Buchanan
10249,Toms Spezialitäten,Michael Suyama
10250,Hanari Carnes,Margaret Peacock
10251,Victuailles en stock,Janet Leverling
10252,Suprêmes délices,Margaret Peacock


Nota: lo siguiente hace que podamos concatenar nombre y apellido para mostrarlo

e.FirstName || ' ' || e.LastName as Empleado

## 4. Funciones de Agregación
Calculemos algunas estadísticas sobre los pedidos:

In [7]:
%%sql
SELECT 
    COUNT(*) as TotalPedidos,
    COUNT(DISTINCT CustomerID) as ClientesUnicos,
    AVG(Freight) as PromedioEnvio,
    MAX(OrderDate) as UltimoPedido
FROM Orders;

 * sqlite:///northwind.db
Done.


TotalPedidos,ClientesUnicos,PromedioEnvio,UltimoPedido
16282,93,248.58558530893012,2023-10-28 00:09:48


## 5. GROUP BY
Veamos el total de ventas por categoría:

In [11]:
%%sql
SELECT 
    c.CategoryName,
    COUNT(DISTINCT p.ProductID) as CantidadProductos,
    SUM(od.Quantity * od.UnitPrice) as VentasTotal
FROM Categories c
JOIN Products p ON c.CategoryID = p.CategoryID
JOIN "Order Details" od ON p.ProductID = od.ProductID
GROUP BY c.CategoryID, c.CategoryName
ORDER BY VentasTotal DESC;

 * sqlite:///northwind.db
Done.


CategoryName,CantidadProductos,VentasTotal
Beverages,12,92181842.95
Confections,13,66347544.94
Meat/Poultry,6,64896314.410000004
Dairy Products,10,58034940.0
Condiments,12,55802774.45
Seafood,12,49931965.52
Produce,5,32706403.9
Grains/Cereals,7,28573512.55


Nota: Tenemos que usar "Order Details" entre comillas porque la tabla tiene un espacio en el nombre.

# Ejercicios

Vamos ahora a dejar unos ejercicios para que ustedes hagan. Intenten resolver los ejercicios por su cuenta. Las respuesta se encuentran más al final del notebook.

1. Mostrar los 5 productos más vendidos (por cantidad de unidades) junto con su categoría.

In [ ]:
%%sql


2. Encontrar los clientes que no han realizado ningún pedido desde agosto de 2023.

In [ ]:
%%sql


3. Listar los empleados que han vendido más de $100,000 en total, mostrando el monto total vendido.

In [ ]:
%%sql


4. Mostrá los empleados que atendieron al menos una orden con destino a un cliente en Alemania, junto con la cantidad de esas órdenes.

In [ ]:
%%sql


5. Calcular el valor total del inventario (unidades en stock * precio unitario) por categoría.


In [ ]:
%%sql


6. Listar los empleados con la cantidad total de órdenes que gestionaron y el total de productos vendidos en esas órdenes.

In [ ]:
%%sql


7. Para cada proveedor (Suppliers), mostrá cuántos productos distintos suyos fueron vendidos al menos una vez.

In [ ]:
%%sql


8. Clientes por país con total de órdenes y unidades compradas

In [ ]:
%%sql


9. Promedio de productos por orden, por empleado

In [ ]:
%% sql


10. Mostrar los productos vendidos (distintos) por categoría

In [ ]:
%% sql


# Soluciones

A continuación se encuentran las soluciones para los ejercicios que se plantearon arriba. Intenten resolverlos por su cuenta antes de ver las respuestas.

1. Los 5 productos más vendidos

In [13]:
%%sql
SELECT p.ProductName, c.CategoryName, SUM(od.Quantity) as UnidadesVendidas
FROM Products p
JOIN Categories c ON p.CategoryID = c.CategoryID
JOIN "Order Details" od ON p.ProductID = od.ProductID
GROUP BY p.ProductID, p.ProductName, c.CategoryName
ORDER BY UnidadesVendidas DESC
LIMIT 5;

 * sqlite:///northwind.db
Done.


ProductName,CategoryName,UnidadesVendidas
Louisiana Hot Spiced Okra,Condiments,206213
Sir Rodney's Marmalade,Confections,205637
Teatime Chocolate Biscuits,Confections,205487
Sirop d'érable,Condiments,205005
Gumbär Gummibärchen,Confections,204761


2. Clientes sin pedidos desde agosto de 2023

In [24]:
%%sql
SELECT c.CompanyName, c.Country, MAX(o.OrderDate) as UltimoPedido
FROM Customers c
LEFT JOIN Orders o ON c.CustomerID = o.CustomerID
GROUP BY c.CustomerID, c.CompanyName, c.Country
HAVING UltimoPedido < '2023-08-01' OR UltimoPedido IS NULL;

 * sqlite:///northwind.db
Done.


CompanyName,Country,UltimoPedido
FISSA Fabrica Inter. Salchichas S.A.,Spain,2023-05-11 09:30:59
Mère Paillarde,Canada,2023-07-09 22:36:22
Pericles Comidas clásicas,Mexico,2023-07-27 03:20:11
QUICK-Stop,Germany,2023-06-22 00:56:44
The Big Cheese,USA,2023-07-16 19:29:58


3. Empleados con ventas mayores a $100,000

In [26]:
%%sql
SELECT 
    e.FirstName || ' ' || e.LastName as Empleado,
    SUM(od.Quantity * od.UnitPrice) as VentasTotal
FROM Employees e
JOIN Orders o ON e.EmployeeID = o.EmployeeID
JOIN "Order Details" od ON o.OrderID = od.OrderID
GROUP BY e.EmployeeID, e.FirstName, e.LastName
HAVING VentasTotal > 100000
ORDER BY VentasTotal DESC;

 * sqlite:///northwind.db
Done.


Empleado,VentasTotal
Margaret Peacock,51505691.8
Steven Buchanan,51393234.57
Janet Leverling,50455812.22
Nancy Davolio,49669459.34
Robert King,49668627.06
Laura Callahan,49287575.56
Michael Suyama,49144251.53
Anne Dodsworth,49025334.37
Andrew Fuller,48325312.27


4. Mostrá los empleados que atendieron al menos una orden con destino a un cliente en Alemania, junto con la cantidad de esas órdenes.

In [36]:
%%sql
SELECT 
    e.FirstName || ' ' || e.LastName AS Empleado,
    COUNT(o.OrderID) AS OrdenesAlemania
FROM Employees e
JOIN Orders o ON e.EmployeeID = o.EmployeeID
JOIN Customers c ON o.CustomerID = c.CustomerID
WHERE c.Country = 'Germany'
GROUP BY e.EmployeeID, Empleado
ORDER BY OrdenesAlemania DESC;

 * sqlite:///northwind.db
Done.


Empleado,OrdenesAlemania
Andrew Fuller,229
Laura Callahan,229
Margaret Peacock,224
Janet Leverling,218
Nancy Davolio,215
Michael Suyama,215
Anne Dodsworth,198
Robert King,189
Steven Buchanan,178


5. Valor total del inventario por categoría

In [29]:
%%sql
SELECT 
    c.CategoryName,
    SUM(p.UnitsInStock * p.UnitPrice) as ValorInventario
FROM Categories c
JOIN Products p ON c.CategoryID = p.CategoryID
GROUP BY c.CategoryID, c.CategoryName
ORDER BY ValorInventario DESC;

 * sqlite:///northwind.db
Done.


CategoryName,ValorInventario
Seafood,13010.35
Beverages,12480.25
Condiments,12023.55
Dairy Products,11271.2
Confections,10392.199999999999
Meat/Poultry,5729.45
Grains/Cereals,5594.5
Produce,3549.3500000000004


6. Listar los empleados con la cantidad total de órdenes que gestionaron y el total de productos vendidos en esas órdenes.

In [33]:
%%sql
SELECT 
    e.FirstName || ' ' || e.LastName AS Empleado,
    COUNT(DISTINCT o.OrderID) AS TotalOrdenes,
    SUM(od.Quantity) AS TotalProductosVendidos
FROM Employees e
JOIN Orders o ON e.EmployeeID = o.EmployeeID
JOIN "Order Details" od ON o.OrderID = od.OrderID
GROUP BY e.EmployeeID, Empleado
ORDER BY TotalProductosVendidos DESC;


 * sqlite:///northwind.db
Done.


Empleado,TotalOrdenes,TotalProductosVendidos
Margaret Peacock,1908,1783482
Steven Buchanan,1804,1774436
Janet Leverling,1846,1737784
Nancy Davolio,1846,1728186
Robert King,1789,1719478
Laura Callahan,1798,1718574
Michael Suyama,1754,1694483
Anne Dodsworth,1766,1694072
Andrew Fuller,1771,1688107


7. Para cada proveedor (Suppliers), mostrá cuántos productos distintos suyos fueron vendidos al menos una vez.

In [35]:
%%sql
SELECT 
    s.CompanyName AS Proveedor,
    COUNT(DISTINCT p.ProductID) AS ProductosVendidos
FROM Suppliers s
JOIN Products p ON s.SupplierID = p.SupplierID
JOIN "Order Details" od ON p.ProductID = od.ProductID
GROUP BY s.SupplierID, s.CompanyName
ORDER BY ProductosVendidos DESC;


 * sqlite:///northwind.db
Done.


Proveedor,ProductosVendidos
"Pavlova, Ltd.",5
Plutzer Lebensmittelgroßmärkte AG,5
New Orleans Cajun Delights,4
"Specialty Biscuits, Ltd.",4
Exotic Liquids,3
Grandma Kelly's Homestead,3
Tokyo Traders,3
Mayumi's,3
Heli Süßwaren GmbH & Co. KG,3
Formaggi Fortini s.r.l.,3


8. Clientes por país con total de órdenes y unidades compradas

In [38]:
%%sql
SELECT 
    c.Country,
    COUNT(DISTINCT o.OrderID) AS TotalOrdenes,
    SUM(od.Quantity) AS TotalUnidades
FROM Customers c
JOIN Orders o ON c.CustomerID = o.CustomerID
JOIN "Order Details" od ON o.OrderID = od.OrderID
GROUP BY c.Country
ORDER BY TotalUnidades DESC;

 * sqlite:///northwind.db
Done.


Country,TotalOrdenes,TotalUnidades
USA,2280,2175993
France,1909,1845501
Germany,1895,1774812
Brazil,1659,1591279
UK,1234,1170811
Mexico,898,861516
Spain,849,830094
Venezuela,726,668721
Canada,520,495139
Argentina,516,489067


9. Promedio de productos por orden, por empleado

In [40]:
%%sql
SELECT 
    e.FirstName || ' ' || e.LastName AS Empleado,
    COUNT(DISTINCT o.OrderID) AS TotalOrdenes,
    SUM(od.Quantity) AS TotalProductos,
    ROUND(1.0 * SUM(od.Quantity) / COUNT(DISTINCT o.OrderID), 2) AS PromedioPorOrden
FROM Employees e
JOIN Orders o ON e.EmployeeID = o.EmployeeID
JOIN "Order Details" od ON o.OrderID = od.OrderID
GROUP BY e.EmployeeID, Empleado
ORDER BY PromedioPorOrden DESC;

 * sqlite:///northwind.db
Done.


Empleado,TotalOrdenes,TotalProductos,PromedioPorOrden
Steven Buchanan,1804,1774436,983.61
Michael Suyama,1754,1694483,966.07
Robert King,1789,1719478,961.14
Anne Dodsworth,1766,1694072,959.27
Laura Callahan,1798,1718574,955.83
Andrew Fuller,1771,1688107,953.19
Janet Leverling,1846,1737784,941.38
Nancy Davolio,1846,1728186,936.18
Margaret Peacock,1908,1783482,934.74


10. Mostrar los productos vendidos (distintos) por categoría

In [42]:
%%sql
SELECT 
    c.CategoryName,
    COUNT(DISTINCT p.ProductID) AS ProductosVendidos
FROM Categories c
JOIN Products p ON c.CategoryID = p.CategoryID
JOIN "Order Details" od ON p.ProductID = od.ProductID
GROUP BY c.CategoryID, c.CategoryName
ORDER BY ProductosVendidos DESC;

 * sqlite:///northwind.db
Done.


CategoryName,ProductosVendidos
Confections,13
Beverages,12
Condiments,12
Seafood,12
Dairy Products,10
Grains/Cereals,7
Meat/Poultry,6
Produce,5
